In [1]:
import mne
#%gui qt
#import matplotlib

#%matplotlib qt
import numpy as np
from matplotlib import pyplot as plt
from stormdb.access import Query
from pickle import load
from scipy import stats
from mne.datasets import sample
from mne.stats import spatio_temporal_cluster_1samp_test
import os
import pickle
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# os.environ['ETS_TOOLKIT'] = 'qt4'
# os.environ['QT_API'] = 'pyqt5'
# %gui qt
#mne.viz.set_3d_backend("notebook")

In [2]:
proj_name = 'MINDLAB2020_MEG-AuditoryPatternRecognition'
wdir = '/projects/' + proj_name + '/scratch/working_memory/'
stats_dir = wdir + 'results/stats/'
data_dir = wdir + 'averages/data/'
figures_dir = wdir + 'results/figures/'

subs_dir = '/projects/' + proj_name + '/scratch/fs_subjects_dir/'
sample_path = sample.data_path()
sample_subjects_dir = sample_path + '/subjects'
# src_sample = mne.read_source_spaces(sample_subjects_dir +
#                                 '/fsaverage/bem/fsaverage-vol-5-src.fif')
src_sample = mne.read_source_spaces(subs_dir +
                                'fsaverage/bem/fsaverage-vol-5-src.fif')
#label_file = subs_dir + 'fsaverage/mri/aparc.DKTatlas+aseg.mgz'
label_file = sample_subjects_dir + '/fsaverage/mri/aparc.a2009s+aseg.mgz'


    Reading a source space...
    [done]
    1 source spaces read


In [3]:
# Read stats
periods = {'encoding': [0,2],'delay': [2,4]}#,'retrieval': [4,6.25]}
#periods = {'delay': [2,4]}#,'retrieval': [4,6.25]}

stats = {}

for p in periods:
    csfname = '{}ERF_source_stats_{}.p'.format(stats_dir, p)
    cfile = open(csfname,'rb')
    stats[p] = pickle.load(cfile)
    cfile.close()
print(stats)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
for c in stats:
    print(c)
    for s in stats[c]:
        print(s)

encoding
maintenance/mel1
maintenance/mel2
manipulation/mel1
manipulation/mel2
melody_maintenance
melody_manipulation
block
melody
interaction
delay
maintenance/mel1
maintenance/mel2
manipulation/mel1
manipulation/mel2
melody_maintenance
melody_manipulation
block
melody
interaction


In [5]:
# Load and morph a source time course
dfname = data_dir + '0021_LZW/0021_LZW_evoked_sources_lf_0.05_hf_40_tstep_0.025_twin_0.05.p'
dfile = open(dfname,'rb')
all_stc = pickle.load(dfile)
dfile.close()
stc = deepcopy(all_stc['maintenance'])
del all_stc

# load source morph
morph = mne.read_source_morph(subs_dir + '0021_LZW/bem/0021_LZW_vol-morph.h5')
stc = morph.apply(stc)
print(stc)

MemoryError: 

In [ ]:
 labels = mne.get_volume_labels_from_aseg(label_file)

In [ ]:
for l in labels:
    print(l)

In [ ]:
clabels = ['Left-Thalamus-Proper',
           'Right-Thalamus-Proper',
           #'Left-Caudate',
           #'Right-Caudate',
           'ctx_lh_G_and_S_cingul-Ant',
           'ctx_rh_G_and_S_cingul-Ant',
           'ctx_lh_G_and_S_cingul-Mid-Ant',
           'ctx_rh_G_and_S_cingul-Mid-Ant',
           'ctx_lh_G_and_S_cingul-Mid-Post',
           'ctx_rh_G_and_S_cingul-Mid-Post',
           'ctx_lh_G_temp_sup-G_T_transv',
           'ctx_rh_G_temp_sup-G_T_transv',
           'ctx_lh_G_temp_sup-Lateral',
           'ctx_rh_G_temp_sup-Lateral']

stc_labels = {}
se_labels = {}
conds = ['maintenance/mel1','maintenance/mel2','manipulation/mel1','manipulation/mel2',
         'melody_maintenance','melody_manipulation','block','melody','interaction']
for c in conds:
    cdata = stc.copy().crop(0,4)
    cse = stc.copy().crop(0,4)
    cdata.data = np.hstack((stats['encoding'][c]['data_mean'],stats['delay'][c]['data_mean'])) # * stats_results[c]['mask'] 
    cse.data = np.hstack((stats['encoding'][c]['data_sd'],stats['delay'][c]['data_sd'])) / np.sqrt(stats['encoding'][c]['n'])# * stats_results[c]['mask'] 
#    cdata.data = stats['encoding'][c]['data_mean'].T # * stats_results[c]['mask'] 
#    cse.data = stats['encoding'][c]['data_sd'].T / np.sqrt(stats['encoding'][c]['n'])# * stats_results[c]['mask'] 
    stc_labels[c] = cdata.extract_label_time_course(labels = [label_file,clabels], src = src_sample, mode = 'auto')
    se_labels[c] = cse.extract_label_time_course(labels = [label_file,clabels], src = src_sample, mode = 'auto')

In [ ]:
times = stc.copy().crop(-0.025,4).times 
periods = ['encoding','delay']
blocks = ['maintenance','manipulation']
cur_labs = ['Right A1','Right cingulate']
lab_ix = [9,7]
titles = ['maintenance','manipulation']
sidx = -1
fig, axes = plt.subplots(ncols=2,nrows=2, figsize = (10,8))

for clix, cl in enumerate(cur_labs):
    for bidx,b in enumerate(blocks):
        lidx = lab_ix[clix]
        sidx = sidx + 1
        rix, cix = sidx//2,sidx%2
        ci_upper1 = stc_labels[b + '/' + 'mel1'][lidx] + 1.96*se_labels[b + '/' + 'mel1'][lidx]
        ci_lower1 = stc_labels[b + '/' + 'mel1'][lidx] - 1.96*se_labels[b + '/' + 'mel1'][lidx]
        ci_upper2 = stc_labels[b + '/' + 'mel2'][lidx] + 1.96*se_labels[b + '/' + 'mel2'][lidx]
        ci_lower2 = stc_labels[b + '/' + 'mel2'][lidx] - 1.96*se_labels[b + '/' + 'mel2'][lidx]
        axes[rix, cix].fill_between(times, ci_lower1, ci_upper1, color='b', alpha=.1)
        axes[rix, cix].fill_between(times, ci_lower2, ci_upper2, color='r', alpha=.1)
        axes[rix, cix].plot(times,stc_labels[b + '/' + 'mel1'][lidx],color='b')
        axes[rix, cix].plot(times,stc_labels[b + '/' + 'mel2'][lidx],color='r')
        #axes[rix, cix].plot(times,stc_labels[b + '/' + 'mel1-mel2'][lidx],color='k')
        axes[rix, cix].set_title(titles[bidx] + ' ' + cl)
        axes[rix, cix].set_ylim(-1.5,1.5)
        axes[rix, cix].set_xlim(0,4)
        axes[rix, cix].set_xlabel('time (s)')
        axes[rix, cix].set_ylabel('source activation (a.u.)')
        axes[rix, cix].axhline(0., color='k')
        axes[rix, cix].axvline(0., color='k')
        axes[rix, cix].axvline(2., color='k',linestyle='--')
        axes[rix, cix].axvline(.5, color='k',linestyle=':')
        axes[rix, cix].axvline(1, color='k',linestyle=':')
        #axes[rix, cix].legend([clabels[cls] for cls in [0,8]])
        axes[rix, cix].legend(['mel1','mel2'])#,'difference'])
plt.tight_layout()
plt.savefig(figures_dir + 'mels_sources_labels.pdf',orientation='portrait')

In [ ]:
times = stc.copy().crop(-.025,4).times 
periods = ['encoding','delay']
blocks = ['maintenance','manipulation']
cur_labs = ['Right A1','Right cingulate']
lab_ix = [9,7]
titles = ['maintenance','manipulation']
sidx = -1
fig, axes = plt.subplots(ncols=2,nrows=2, figsize = (12.5,10))

for clix, cl in enumerate(cur_labs):
    for bidx,b in enumerate(blocks):
        lidx = lab_ix[clix]
        sidx = sidx + 1
        rix, cix = sidx//2,sidx%2
        #axes[rix, cix].plot(times,stc_labels[b + '/' + 'mel1'][lidx],color='b')
        #axes[rix, cix].plot(times,stc_labels[b + '/' + 'mel2'][lidx],color='r')
        ci_upper = stc_labels['melody_' + b][lidx] + 1.96*se_labels['melody_' + b][lidx]
        ci_lower = stc_labels['melody_' + b][lidx] - 1.96*se_labels['melody_' + b][lidx]
        axes[rix, cix].fill_between(times, ci_lower, ci_upper, color='k', alpha=.2)
        axes[rix, cix].plot(times,stc_labels['melody_' + b][lidx],color='k')
        axes[rix, cix].set_title(titles[bidx] + ' ' + cl)
        axes[rix, cix].set_ylim(-.5,.5)
        axes[rix, cix].set_xlim(0,4)
        axes[rix, cix].set_xlabel('time (s)')
        axes[rix, cix].set_ylabel('source activation (a.u.)')
        axes[rix, cix].axhline(0., color='k')
        axes[rix, cix].axvline(0., color='k')
        axes[rix, cix].axvline(2., color='k',linestyle='--')
        axes[rix, cix].axvline(.5, color='k',linestyle=':')
        axes[rix, cix].axvline(1, color='k',linestyle=':')
        #axes[rix, cix].legend([clabels[cls] for cls in [0,8]])
        axes[rix, cix].legend(['mel1 - mel2'])#,'difference'])
plt.tight_layout()
plt.savefig(figures_dir + 'mels_sources_labels_diff.pdf',orientation='portrait')

In [ ]:
# # Plot 1 stc
# p = 'delay'
# c = 'maint'
conds =['maintenance/mel1','maintenance/mel2','manipulation/mel1','manipulation/mel2',
         'melody_maintenance','melody_manipulation','block','melody','interaction']
for p in stats:
    for c in conds:#stats[p]:
        
        #mask = stats[p][c]['pvals'] <= 0.001#25
        mask = stats[p][c]['mask']# <= 0.025
        cstc = stc.copy()
        cstc.data = stats[p][c]['tvals']*mask#*stats[p][c]['mask']
        #stc.tmin = periods[p][0]
#         plot_times = np.arange(0,2,0.1)
#         for pt in plot_times:
        print('{} {} {}'.format(p,c,cstc.tmin))#+pt))
        brain = cstc.mean().plot(subjects_dir=subs_dir,clim={'kind': 'value', 'lims': [0.001,1.9,5]},#time_viewer=True,
                             subject='fsaverage', mode = 'glass_brain',src=src_sample)#,initial_time = stc.tmin+pt)

In [ ]:
# # Plot 1 stc
# for p in stats:
#     for c in stats[p]:
#         mask = stats[p][c]['qvals'] <= 0.025
#         stc.data = stats[p][c]['data_mean']*mask#*stats[p][c]['mask']
#         stc.tmin = periods[p][0]
# #         plot_times = np.arange(0,2,0.1)
# #         for pt in plot_times:
#         print('{} {}'.format(p,c))#,stc.tmin+pt))
#         brain = stc.mean().plot(subjects_dir=subs_dir,clim={'kind': 'value', 'pos_lims': [0.001,0.01,0.2]},#time_viewer=True,
#                              subject='fsaverage', mode = 'stat_map',src=src_sample, initial_pos = [0.03,0,-0.03])#,initial_time = stc.tmin+pt)